# Regression Week 2: Multiple Regression (Interpretation)

The goal of this first notebook is to explore multiple regression and feature engineering with existing Turi Create functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Use SFrames to do some feature engineering
* Use built-in Turi Create functions to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

# Fire up Turi Create

In [79]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [80]:
sales = pd.read_csv('kc_house_data.csv')

# Split data into training and testing.
We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set a random seed (or let Turi Create pick a random seed for you).  

In [81]:
train_data,test_data = train_test_split(
    sales,
    test_size=0.2,  # Equivalent to Turi Create's splitting ratio (1 - 0.8)
    random_state=0  # Equivalent to Turi Create's seed for reproducibility
)

train_data = pd.read_csv('kc_house_train_data.csv')
train_test = pd.read_csv('kc_house_test_data.csv')

# Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

(Aside: We set validation_set = None to ensure that the results are always the same)

In [82]:
example_features = ['sqft_living', 'bedrooms', 'bathrooms']
example_model = LinearRegression().fit(sales[['sqft_living', 'bedrooms', 'bathrooms']], sales['price'])

Now that we have fitted the model we can extract the regression weights (coefficients) as an SFrame as follows:

In [83]:
print(f'{len(example_model.coef_)} coefficients: {example_model.coef_}, \
        intercept:{example_model.intercept_}')

3 coefficients: [   309.39239013 -57860.8943206    7932.71222266],         intercept:74847.14080128458


# Making Predictions

In the gradient descent notebook we use numpy to do our regression. In this book we will use existing Turi Create functions to analyze multiple regressions. 

Recall that once a model is built we can use the .predict() function to find the predicted values for data we pass. For example using the example model above:

In [84]:
example_predictions = example_model.predict(train_data[example_features])
print(example_predictions) # should be 271789.505878

[274280.19041522 714251.50297385 205290.20478263 ... 394466.59529462
 416124.0626037  280655.1242594 ]


# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [85]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions
    y_predict = model.predict(data)
    
    # Then compute the residuals/errors
    rs = outcome - y_predict
    
    # Then square and add them up
    RSS = sum(rs * rs)
    
    return(RSS)    

Test your function by computing the RSS on TEST data for the example model:

In [86]:
rss_example_train = get_residual_sum_of_squares(example_model, test_data[example_features], test_data['price'])
print(rss_example_train) # should be 2.7376153833e+14

258550070119474.53


# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

In [87]:
from math import log

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
As an example here's the first one:

In [88]:
train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2)

In [89]:
# create the remaining 3 features in both TEST and TRAIN data
train_data['bed_bath_rooms'] = train_data['bedrooms'] * train_data['bathrooms']
test_data['bed_bath_rooms'] = test_data['bedrooms'] * test_data['bathrooms']

train_data['log_sqft_living'] = list(map(log, train_data['sqft_living']))
test_data['log_sqft_living'] = list(map(log, test_data['sqft_living']))

train_data['lat_plus_long'] = train_data['lat'] + train_data['long']
test_data['lat_plus_long'] = test_data['lat'] + test_data['long']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

**Quiz Question: What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)**

In [90]:
import numpy as np

np.mean((test_data['bed_bath_rooms']))

np.float64(7.447721489706223)

# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [91]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Now that you have the features, learn the weights for the three different models for predicting target = 'price' using turicreate.linear_regression.create() and look at the value of the weights/coefficients:

In [92]:
# Learn the three models: (don't forget to set validation_set = None)
model_1 = LinearRegression().fit(train_data[model_1_features], train_data['price'])

model_2 = LinearRegression().fit(train_data[model_2_features], train_data['price'])

model_3 = LinearRegression().fit(train_data[model_3_features], train_data['price'])

In [93]:
# Examine/extract each model's coefficients:
print(f'{len(model_1.coef_)} coefficients: {model_1.coef_}, \
        intercept:{model_1.intercept_}')

print(f'{len(model_2.coef_)} coefficients: {model_2.coef_}, \
        intercept:{model_2.intercept_}')

print(f'{len(model_3.coef_)} coefficients: {model_3.coef_}, \
        intercept:{model_3.intercept_}')

5 coefficients: [ 3.12258646e+02 -5.95865332e+04  1.57067421e+04  6.58619264e+05
 -3.09374351e+05],         intercept:-69075726.79256989
6 coefficients: [ 3.06610053e+02 -1.13446368e+05 -7.14613083e+04  6.54844630e+05
 -2.94298969e+05  2.55796520e+04],         intercept:-66867968.8710789
9 coefficients: [ 5.29422820e+02  3.45142296e+04  6.70607813e+04  5.34085611e+05
 -4.06750711e+05 -8.57050439e+03 -6.78858667e+03 -5.61831484e+05
  1.27334900e+05],         intercept:-62036084.986098334


**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?**
1.57067421e+04

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?**
-7.14613083e+04

Think about what this means.

# Comparing multiple models

Now that you've learned three models and extracted the model weights we want to evaluate which model is best.

First use your functions from earlier to compute the RSS on TRAINING Data for each of the three models.

In [102]:
# Compute the RSS on TRAINING data for each of the three models and record the values:
rss_model_1_train = get_residual_sum_of_squares(model_1, train_data[model_1_features], train_data['price'])
print(rss_model_1_train)

rss_model_2_train = get_residual_sum_of_squares(model_2, train_data[model_2_features], train_data['price'])
print(rss_model_2_train)

rss_model_3_train = get_residual_sum_of_squares(model_3, train_data[model_3_features], train_data['price'])
print(rss_model_3_train)

967879963049548.1
958419635074072.0
903436455050480.2


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TRAINING Data?** Is this what you expected?

Now compute the RSS on on TEST data for each of the three models.

In [104]:
# Compute the RSS on TESTING data for each of the three models and record the values:
rss_model_1_test = get_residual_sum_of_squares(model_1, test_data[model_1_features], test_data['price'])
print(rss_model_1_test)

rss_model_2_test = get_residual_sum_of_squares(model_2, test_data[model_2_features], test_data['price'])
print(rss_model_2_test)

rss_model_3_test = get_residual_sum_of_squares(model_3, test_data[model_3_features], test_data['price'])
print(rss_model_3_test)

212800313012877.16
210121081660072.7
202958529940651.22


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TESTING Data?** Is this what you expected? Think about the features that were added to each model from the previous.

In [ ]:
# Logically lat_plus_long has no clear indication to influence the model.